# SQL Querying

This notebook can be used to query tables in the Congressional Data database. In order to use this notebook, you will need to set an environment variable 'CD_DWH' to the database connection string. If you do not have the credentials, please slack us at #datasci-congressdata channel and/or talk to a project lead.

**It is best practice to not hard code database URI strings directly in notebooks or code as when we push to Github, that would mean credentials are public for anyone to see.**

In [ ]:
import os
import sys

import pandas as pd
pd.options.display.max_columns = 999
import sqlalchemy as sqla
from sqlalchemy import create_engine

from plotnine import *
import math

DB_URI = os.getenv('CD_DWH')
engine = create_engine(DB_URI)

In [ ]:
# Checking that the Kernel is using the Conda environment datasci-congressional-data
# Below you should see something like '/Users/Username/anaconda3/envs/datasci-congressional-data/bin/python
# If you do NOT see "datasci-congressional-data" this means you are not in the right Python Environment
# Please make sure you have gone through the onboarding docs and/or talk to a project lead.
sys.executable

Below are the tables that currently exist in the database!

## Query Factors & visualize frequency plots

In [ ]:
QUERRY = """
SELECT
    recipient_candidate_name
    , recipient_candidate_office
    , donor_name
    , donor_organization
    , transaction_amount
    , transaction_date
  FROM trg_analytics.candidate_contributions
  GROUP BY
    recipient_candidate_office, transaction_id"""
with engine.begin() as conn:
    results = pd.read_sql(QUERRY, conn)

In [ ]:
results.set_index(pd.PeriodIndex(results['transaction_date'], freq='Y'),inplace=True,drop=True)
results.head()

In [ ]:
candidate_contrib = results.groupby('recipient_candidate_office').agg({'transaction_amount': ['sum']})
candidate_contrib_vals = candidate_contrib['transaction_amount', 'sum']
candidate_contrib_vals.sort_values().plot.barh()

(ggplot(data = results)
  + geom_freqpoly(aes(x=results['transaction_date']
                      , color=results['recipient_candidate_office'])
                      , binwidth=2000.0)
)

In [ ]:
st_assem_results = results[results['recipient_candidate_office']=='State Assembly']
st_assem_candid = st_assem_results.groupby('recipient_candidate_name').agg({'transaction_amount': ['sum']})
st_assem_candid_vals = st_assem_candid['transaction_amount','sum']
st_assem_candid_vals.sort_values(ascending=False)[:30].plot.bar()

In [ ]:
donor_contrib = st_assem_results.groupby('donor_organization').agg({'transaction_amount': ['sum']})
donor_contrib_vals = donor_contrib['transaction_amount', 'sum']
donor_contrib_vals.sort_values(ascending=False).plot.bar()

In [ ]:
ind_results = st_assem_results[results['donor_organization']=='COM']
